<a href="https://colab.research.google.com/github/RoshniBanu/SDC_Codes_Colab/blob/main/Langchain%2BRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

YOUTUBE VIDEO SUMMARIZER WITH LANGCHAIN AND RAG

In [11]:
!pip install langchain faiss-cpu youtube-transcript-api sentence-transformers transformers


In [22]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain.embeddings.base import Embeddings
import re

# 1. Custom wrapper for SentenceTransformer to work with LangChain
class LocalEmbedding(Embeddings):
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts)

    def embed_query(self, text):
        return self.model.encode([text])[0]

# 2. Get transcript from YouTube video
def get_transcript(video_url):
    video_id = re.search(r"(?<=v=)[^&#]+", video_url)
    if not video_id:
        raise ValueError("Invalid YouTube URL")
    video_id = video_id.group(0)
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    text = " ".join([entry["text"] for entry in transcript])
    return text

# 3. Split long text into smaller chunks
def split_text_into_docs(text, chunk_size=1000, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.create_documents([text])

# 4. Create FAISS vector store with local embeddings
def create_vectorstore(docs):
    embedding = LocalEmbedding()
    db = FAISS.from_documents(docs, embedding)
    return db

# 5. Retrieve relevant chunks and summarize
def summarize_retrieved_docs(vectorstore, query="Summarize the video"):
    retriever = vectorstore.as_retriever()
    docs = retriever.get_relevant_documents(query)
    combined_text = " ".join([doc.page_content for doc in docs])

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(combined_text, max_length=300, min_length=50, do_sample=False)[0]["summary_text"]
    return summary

# 6. Complete pipeline
def summarize_youtube_video(video_url):
    print("📥 Getting transcript...")
    text = get_transcript(video_url)

    print("✂️ Splitting into chunks...")
    docs = split_text_into_docs(text)

    print("📚 Building vectorstore...")
    vectorstore = create_vectorstore(docs)

    print("🧠 Summarizing...")
    summary = summarize_retrieved_docs(vectorstore)

    return summary

# 7. Run it!
video_url = "https://www.youtube.com/watch?v=d4yCWBGFCEs"  # Replace with your video
summary = summarize_youtube_video(video_url)
print("\n📋 Summary:\n", summary)


📥 Getting transcript...
✂️ Splitting into chunks...
📚 Building vectorstore...
🧠 Summarizing...


Device set to use cpu



📋 Summary:
 This tool is ready now it's going to be very useful to my equity research analyst my peter Panda who is investing on aoki's behalf because now you don't have to read so many articles whatever question you have you can ask it to this news research tool it will not only give you the answer but it gives you the sources reference. By the way with llm boom many clients are building this kind of text to or video model you can just Google open as s you'll find the demo.


YOUTUBE VIDEO SUMMARIZER WITH URL INPUT WITH LANGCHAIN AND RAG

In [13]:
!pip install langchain faiss-cpu youtube-transcript-api sentence-transformers transformers ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00


In [23]:
import ipywidgets as widgets
from IPython.display import display
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain.embeddings.base import Embeddings
import re

# 1. Custom wrapper for SentenceTransformer to work with LangChain
class LocalEmbedding(Embeddings):
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts)

    def embed_query(self, text):
        return self.model.encode([text])[0]

# 2. Get transcript from YouTube video
def get_transcript(video_url):
    video_id = re.search(r"(?<=v=)[^&#]+", video_url)
    if not video_id:
        raise ValueError("Invalid YouTube URL")
    video_id = video_id.group(0)
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    text = " ".join([entry["text"] for entry in transcript])
    return text

# 3. Split long text into smaller chunks
def split_text_into_docs(text, chunk_size=1000, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.create_documents([text])

# 4. Create FAISS vector store with local embeddings
def create_vectorstore(docs):
    embedding = LocalEmbedding()
    db = FAISS.from_documents(docs, embedding)
    return db

# 5. Retrieve relevant chunks and summarize
def summarize_retrieved_docs(vectorstore, query="Summarize the video"):
    retriever = vectorstore.as_retriever()
    docs = retriever.get_relevant_documents(query)
    combined_text = " ".join([doc.page_content for doc in docs])

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(combined_text, max_length=300, min_length=50, do_sample=False)[0]["summary_text"]
    return summary

# 6. Complete pipeline
def summarize_youtube_video(video_url):
    print("📥 Getting transcript...")
    text = get_transcript(video_url)

    print("✂️ Splitting into chunks...")
    docs = split_text_into_docs(text)

    print("📚 Building vectorstore...")
    vectorstore = create_vectorstore(docs)

    print("🧠 Summarizing...")
    summary = summarize_retrieved_docs(vectorstore)

    return summary

# 7. Interactive Widgets for URL Input
def on_button_click(b):
    video_url = url_textbox.value
    if not video_url:
        print("Please provide a valid URL.")
        return
    print("⌛ Processing the video...")
    summary = summarize_youtube_video(video_url)
    print("\n📋 Summary:\n", summary)

# Create a URL input field
url_textbox = widgets.Text(
    description='YouTube URL:',
    placeholder='Enter YouTube video URL',
    layout=widgets.Layout(width='50%')
)

# Create a button to trigger the summarization
summarize_button = widgets.Button(description="Summarize Video", layout=widgets.Layout(width='20%'))

# Bind button click to the function
summarize_button.on_click(on_button_click)

# Display the input field and button
display(url_textbox, summarize_button)


Text(value='', description='YouTube URL:', layout=Layout(width='50%'), placeholder='Enter YouTube video URL')

Button(description='Summarize Video', layout=Layout(width='20%'), style=ButtonStyle())

⌛ Processing the video...
📥 Getting transcript...
✂️ Splitting into chunks...
📚 Building vectorstore...
🧠 Summarizing...


Device set to use cpu



📋 Summary:
 This tool is ready now it's going to be very useful to my equity research analyst my peter Panda who is investing on aoki's behalf because now you don't have to read so many articles whatever question you have you can ask it to this news research tool it will not only give you the answer but it gives you the sources reference. By the way with llm boom many clients are building this kind of text to or video model you can just Google open as s you'll find the demo.


MEDICAL q&a CHATBOT WITH PUBMEN

In [15]:
!pip install requests langchain openai


  Using cached openai-1.74.0-py3-none-any.whl.metadata (25 kB)
Using cached openai-1.74.0-py3-none-any.whl (644 kB)


In [17]:
!pip install requests transformers bs4 sentence-transformers langchain faiss-cpu


In [25]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. Function to search PubMed and retrieve relevant articles
def search_pubmed(query, max_results=5):
    # PubMed API endpoint
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        'db': 'pubmed',
        'term': query,
        'retmax': max_results,
        'retmode': 'xml',
        'sort': 'relevance',
    }

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise Exception("Error searching PubMed")

    # Parse the response and extract article IDs
    soup = BeautifulSoup(response.content, 'xml')
    article_ids = [id_tag.text for id_tag in soup.find_all('Id')]
    return article_ids

# 2. Function to fetch article details from PubMed
def fetch_article_details(article_ids):
    # PubMed article details API endpoint
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    ids = ','.join(article_ids)
    params = {
        'db': 'pubmed',
        'id': ids,
        'retmode': 'xml',
    }

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise Exception("Error fetching article details from PubMed")

    # Parse the response and extract article titles and abstracts
    soup = BeautifulSoup(response.content, 'xml')
    articles = []
    for article in soup.find_all('PubmedArticle'):
        title = article.find('ArticleTitle').text if article.find('ArticleTitle') else 'No title'
        abstract = article.find('AbstractText').text if article.find('AbstractText') else 'No abstract available'
        articles.append(f"Title: {title}\nAbstract: {abstract}")
    return articles

# 3. Function to create a medical Q&A chatbot
def medical_qa_chatbot(query):
    # Search PubMed for relevant articles
    print(f"🔍 Searching PubMed for: {query}")
    article_ids = search_pubmed(query, max_results=5)

    if not article_ids:
        return "Sorry, I couldn't find any relevant articles on PubMed."

    # Fetch article details (titles and abstracts)
    articles = fetch_article_details(article_ids)

    # Combine articles into one document for summarization
    documents = "\n\n".join(articles)

    # Use SentenceTransformer to create embeddings
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode([documents])

    # Create FAISS index to store and retrieve the embeddings
    index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance (Euclidean distance)
    index.add(np.array([embeddings[0]]).astype(np.float32))

    # Use HuggingFace model for question answering
    qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

    # Retrieve relevant documents from FAISS index
    retrieved_text = documents  # In this case, we're directly using the full text

    # Use the question answering model to answer based on the retrieved text
    answer = qa_model(question=query, context=retrieved_text)

    return answer['answer']

# 4. Example usage
query = input("Enter your medical question: ")
response = medical_qa_chatbot(query)
print("\n🔑 Answer:", response)



Enter your medical question: symptoms of flu
🔍 Searching PubMed for: symptoms of flu


Device set to use cpu



🔑 Answer: fever, fatigue, cough, and body aches
